### Imports

In [110]:
import pandas as pd
pd.set_option('display.max_columns', None)
import pytz

import requests as rq
import json
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Keys

In [31]:
# Get demo API key
def get_demo_key():
    f = open("/home/vikas/Documents/CG_demo_key.json")
    key_dict = json.load(f)
    return key_dict["key"]

In [32]:
# Get pro API key
def get_pro_key():
    f = open("/home/vikas/Documents/CG_pro_key.json")
    key_dict = json.load(f)
    return key_dict["key"]

### API status

In [33]:
PUB_URL = "https://api.coingecko.com/api/v3"
PRO_URL = "https://pro-api.coingecko.com/api/v3"

In [79]:
def get_response(endpoint, headers, params, URL):
    
    url = "".join((URL, endpoint))
    response = rq.get(url, headers = headers, params = params)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data, check status code {response.status_code}")

    data = response.json()
    return data

In [35]:
use_demo = {
           "accept": "application/json",
           "x-cg-demo-api-key" : get_demo_key() 
}

use_pro = {
         "accept": "application/json",
         "x-cg-pro-api-key" : get_pro_key()
}

In [36]:
get_response("/ping", use_demo, "", PUB_URL)

{'gecko_says': '(V3) To the Moon!'}

### Create URLs

In [37]:
def get_url(url_type,
            network = "",
            dex = "",
            pool_address = "",
            token_address = ""):

    url_dict = {
        "new_pools": f"/onchain/networks/new_pools",
        "trending_pools": f"/onchain/networks/{network}/pools",
        "top_pools": f"/onchain/networks/{network}/pools",
        "top_pools_dex": f"/onchain/networks/{network}/dexes/{dex}/pools",
        "specific_pool_dex": f"/onchain/networks/{network}/pools/{pool_address}",
        "top_pools_add": f"/onchain/networks/{network}/tokens/{token_address}/pools",
        "token_data": f"/onchain/networks/{network}/tokens/{token_address}",
        "token_info": f"/onchain/networks/{network}/tokens/{token_address}/info"
    }

    return url_dict[url_type]

### Get list of networks

In [38]:
def get_all_networks():    

    networks_list_response = get_response("/onchain/networks",
                                          use_pro, 
                                          "",
                                          PRO_URL)

    return pd.DataFrame(networks_list_response["data"])

In [39]:
df_all_networks = get_all_networks()

In [40]:
df_all_networks[df_all_networks["id"].str.contains("bsc",
                                                   case=False,
                                                   na=False)]

,id,type,attributes
1,bsc,network,"{'name': 'BNB Chain', 'coingecko_asset_platfor..."


### Get top pools for a specific network

In [167]:
# def collect_response(list_response):    

#     response_all = []

#     for response in list_response["data"]:
#         all_attributes = response["attributes"]
#         daily_tx = all_attributes["transactions"]["h24"]
#         rel = response["relationships"]
#         base_token_add = rel["base_token"]["data"]["id"]
                
#         temp_dict = dict(
#             pair = all_attributes["name"],
#             dex = rel["dex"]["data"]["id"],
#             network = rel["network"]["data"]["id"],
#             token_add = base_token_add.split("_")[1],
#             pool_add = all_attributes["address"],
#             fdv_usd = all_attributes["fdv_usd"],
#             market_cap_usd = all_attributes["market_cap_usd"],
#             daily_volume = all_attributes["volume_usd"]["h24"],
#             daily_price_change = all_attributes["price_change_percentage"]["h24"],
#             daily_buys = daily_tx["buys"],
#             daily_sells = daily_tx["sells"],
#             daily_buyers = daily_tx["buyers"],
#             daily_sellers = daily_tx["sellers"]             
#         )
        
#         response_all.append(temp_dict)

#     return response_all

def safe_get(d, path, default=None):
    """Safely get a nested dictionary value."""
    for key in path:
        if isinstance(d, dict) and key in d:
            d = d[key]
        else:
            return default
    return d


def collect_response(list_response):    

    response_all = []

    for response in list_response.get("data", []):
        
        all_attributes = response.get("attributes", {})
        rel = response.get("relationships", {})
        
        # Extract nested values safely
        daily_tx = safe_get(all_attributes, ["transactions", "h24"], {})
        base_token_add = safe_get(rel, ["base_token", "data", "id"], "NA")

        # If token_add exists, split it
        token_add = base_token_add.split("_")[1] if base_token_add != "NA" and "_" in base_token_add else "NA"
        
        temp_dict = dict(
            pair = safe_get(all_attributes, ["name"], "NA"),
            dex = safe_get(rel, ["dex", "data", "id"], "NA"),
            network = safe_get(rel, ["network", "data", "id"], "NA"),
            token_add = token_add,
            pool_add = safe_get(all_attributes, ["address"], "NA"),
            fdv_usd = safe_get(all_attributes, ["fdv_usd"], "NA"),
            market_cap_usd = safe_get(all_attributes, ["market_cap_usd"], "NA"),
            daily_volume = safe_get(all_attributes, ["volume_usd", "h24"], "NA"),
            daily_price_change = safe_get(all_attributes, ["price_change_percentage", "h24"], "NA"),
            daily_buys = safe_get(daily_tx, ["buys"], "NA"),
            daily_sells = safe_get(daily_tx, ["sells"], "NA"),
            daily_buyers = safe_get(daily_tx, ["buyers"], "NA"),
            daily_sellers = safe_get(daily_tx, ["sellers"], "NA")
        )
        
        response_all.append(temp_dict)

    return response_all

In [42]:
def get_top_pools_network(network, sort_by_col):

    target_url = get_url("top_pools", network)

    toppool_list_response = get_response(target_url,
                                         use_pro, 
                                         "",
                                         PRO_URL)

    toppool_all = collect_response(toppool_list_response)   

    return pd.DataFrame(toppool_all).sort_values(by = [f"{sort_by_col}"],
                                                 ascending = False)

In [43]:
get_top_pools_network("solana", "market_cap_usd").head(5)

,pair,dex,token_add,pool_add,fdv_usd,market_cap_usd,daily_volume,daily_price_change,daily_buys,daily_sells,daily_buyers,daily_sellers
17,BRETTA / SOL,raydium,DXBYAw9aQheMdujaLZYnVSpKSK4n8jMS7HfLbiv5RWnS,GVmfWskwEi5AvyULvhLgDmo1mVqrdNAk2qFMfzyTafNo,65870.1872249867,65870.1872249867,1050.4719145304,-10.44,43482,15934,39644,13933
0,USD1 / SOL,raydium-clmm,USD1ttGY1N17NEEHLmELoaybftRBUSErhqYiQzvEmuB,AQAGYQsdU853WAKhXM79CgNdoyhrRwXvYHX6qrDyC1FS,143886301.559806,2733075779.77761,29655212.802627,-0.037,148404,126357,16780,15325
9,mansuki / SOL,pumpswap,JBYXZaWcgdWSGvdYXior1hYyygm3eMYiPdd7KiWcpump,29TXFSSyeqfYVbzwNDktxWV6uZSathP2ZET4EVcjRHxH,172923.74587721,172923.74587721,140713.934389822,-35.475,67533,1480,66869,806
8,pippin / SOL,raydium,Dfh5DzRgSvvCFDoYc2ciTkMrbDfRKybA4SoFbPmApump,8WwcNqdZjCY5Pt7AkhupAFknV2txca9sq6YBkGzLbvdt,166482256.64896,166482262.859258,34170064.3668492,-32.953,41631,28304,1850,1506
10,SOL / USDT 0.01%,raydium-clmm,So11111111111111111111111111111111111111112,3nMFwZXwY1s1M5s8vYAHqd4wGs4iSxXE4LRoUMMYqEgF,1659610926.24182,1659610926.24181,12051433.9066225,-1.844,37519,30616,2527,3653


### Get data for a specific pool address

In [46]:
def collect_pool_response(list_response):    

    response = list_response["data"]
    all_attributes = response["attributes"]
    daily_tx = all_attributes["transactions"]["h24"]
    rel = response["relationships"]
        
    response_dict = dict(
        pair = all_attributes["name"],
        dex = rel["dex"]["data"]["id"],
        add = all_attributes["address"],
        fdv_usd = all_attributes["fdv_usd"],
        market_cap_usd = all_attributes["market_cap_usd"],
        daily_volume = all_attributes["volume_usd"]["h24"],
        daily_price_change = all_attributes["price_change_percentage"]["h24"],
        daily_buys = daily_tx["buys"],
        daily_sells = daily_tx["sells"],
        daily_buyers = daily_tx["buyers"],
        daily_sellers = daily_tx["sellers"]
    )

    return response_dict

In [47]:
def get_pool_data(network, dex, pool_address):

    target_url = get_url("specific_pool_dex", network, dex, pool_address)

    pool_list_response = get_response(target_url,
                                      use_pro, 
                                      "",
                                      PRO_URL)

    pool_all = collect_pool_response(pool_list_response)   

    return pool_all

In [48]:
get_pool_data("solana", "", "FAqh648xeeaTqL7du49sztp9nfj5PjRQrfvaMccyd9cz")

{'pair': 'PENGU / SOL',
 'dex': 'orca',
 'add': 'FAqh648xeeaTqL7du49sztp9nfj5PjRQrfvaMccyd9cz',
 'fdv_usd': '849578861.114266',
 'market_cap_usd': '696072342.725605',
 'daily_volume': '818227.704950673',
 'daily_price_change': '2.09',
 'daily_buys': 1656,
 'daily_sells': 1482,
 'daily_buyers': 104,
 'daily_sellers': 77}

### Get data for a specific token address

In [87]:
def collect_token_response(list_response):    

    response = list_response.get("data", {})
    all_attributes = response.get("attributes", {})
    
    # Safely extract launchpad_details or default to empty dict
    launchpad_details = all_attributes.get("launchpad_details", {})

    response_dict = dict(
        grad_pert = (
            launchpad_details.get("graduation_percentage")
            if launchpad_details else 0
        ),
        completed = launchpad_details.get("completed", False),
        completed_at = launchpad_details.get("completed_at", None),
        dest_pool = launchpad_details.get("migrated_destination_pool_address", None)
    )

    return response_dict

In [81]:
def get_token_data(network, token_address):

    target_url = get_url("token_data",
                         network,
                         "",
                         "",
                         token_address)

    token_list_response = get_response(target_url,
                                  use_pro, 
                                  "",
                                  PRO_URL)

    token_all = collect_token_response(token_list_response)   

    return token_all

### Track tokens

In [108]:
list_of_tokens = [
    "96nXL5td1DxpySaYK8yvkazfteyi6YwqVJYx4w1tpump",
    "BdD7EsVWdNbiM5PZJsBEmZj4VdCrC3myCDTHr2G9moon",
    "7JPBqCzTyVBWJEL79jBvRYBReCKqBSFU1HQyo3jHwave",
    "DhSx8SRk9A3sNze2B2Pk9dnDsfcgQZufsqyWm4qkwave",
    "pHMXKasJ6mrEKJAD5sXREVGkpphLJYjN3yRwWVnpump",
    "x8KtZeEW4WHBk8YhtVmNc3ga3B4GSN1gLDTVKe5wave",
    "5SEjkg4TZx2dPriqvbCCMt6GQCUMXWyhLNR7r6BEpump"
]

In [140]:
def track_status(list_of_tokens, network, threshold):

    all_token_data = []

    for token in list_of_tokens:
        token_data = get_token_data(network, token)
        token_data["token"] = token
        all_token_data.append(token_data)

    df_token = pd.DataFrame(all_token_data)
    
    # Convert to datetime with UTC timezone
    df_token["completed_at"] = pd.to_datetime(
        df_token["completed_at"],
        utc=True,
        errors='coerce'  # Invalid strings will become NaT
    )

    # Convert to CET (Central European Time)
    df_token["completed_at"] = df_token["completed_at"].dt.tz_convert("Europe/Berlin")
    
    # Identify rows with graduation % above a certain threshold
    df_token["above_threshold"] = df_token["grad_pert"] > threshold

    # Move token column to the end
    column_to_move = 'token'

    # Reorder columns
    df_token = (
        df_token[[col for col in df_token.columns
                  if col != column_to_move]
                          + [column_to_move]]
    )

    return df_token

In [141]:
df_status = track_status(list_of_tokens, "solana", 95)

df_status

,grad_pert,completed,completed_at,dest_pool,above_threshold,token
0,0.19,False,NaT,None,False,96nXL5td1DxpySaYK8yvkazfteyi6YwqVJYx4w1tpump
1,0.69,False,NaT,None,False,BdD7EsVWdNbiM5PZJsBEmZj4VdCrC3myCDTHr2G9moon
2,0.07,False,NaT,None,False,7JPBqCzTyVBWJEL79jBvRYBReCKqBSFU1HQyo3jHwave
3,27.85,False,NaT,None,False,DhSx8SRk9A3sNze2B2Pk9dnDsfcgQZufsqyWm4qkwave
4,100.00,True,2025-12-07 17:54:13+01:00,65B2xsKv1w2NdyiPBBy5nDc2V77wZAQkvPw1fwWd9xGX,True,pHMXKasJ6mrEKJAD5sXREVGkpphLJYjN3yRwWVnpump
5,0.00,False,NaT,None,False,x8KtZeEW4WHBk8YhtVmNc3ga3B4GSN1gLDTVKe5wave
6,100.00,True,2025-12-07 18:41:40+01:00,ALoFpEmdVyBQ7HPatdYPpaEJS5DEdBWYC22QVzMafXMg,True,5SEjkg4TZx2dPriqvbCCMt6GQCUMXWyhLNR7r6BEpump


### Notification email

In [56]:
# Get sender password
def get_app_password():
    f = open("/home/vikas/Documents/Gmail_app_pass.json")
    key_dict = json.load(f)
    return key_dict["pass"]

In [98]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_graduation_alert(to_email, subject, html_body):
    
    # Replace with your Gmail email address
    sender_email = 'nftdemoapp@gmail.com'
    
    # Replace with your Gmail app password
    sender_password = get_app_password()

    # Email content
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = to_email
    msg['Subject'] = subject

    # Attach the body of the email
    msg.attach(MIMEText(html_body, 'html'))

    # Establish a connection to the SMTP server
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        
        # Send the email
        server.sendmail(sender_email, to_email, msg.as_string())

In [142]:
def send_email(list_of_tokens, network, threshold):

    df_status = track_status(list_of_tokens, network, threshold)
    
    df_to_send = df_status[df_status.completed == True]   

    html_body = f"""
                <html>
                  <body>
                    <p>Hello,<br><br>
                       Here are the latest graduation alerts:
                    </p>
                    {df_to_send.to_html(index=False, border=1)}
                    <p>Regards,<br>Your Crypto Alert Bot</p>
                  </body>
                </html>
                """

    try:
        send_graduation_alert('vikas.negi10@gmail.com', 'Token Graduation Alert', html_body)
    except Exception as e:
        print(f"Unable to send the alert email: {e}")

In [143]:
send_email(list_of_tokens, "solana", 95)

### Get new pools

In [163]:
def get_new_pools(sort_by_col, num_rows):

    target_url = get_url("new_pools")

    toppool_list_response = get_response(target_url,
                                         use_pro, 
                                         "",
                                         PRO_URL)

    toppool_all = collect_response(toppool_list_response)   

    return pd.DataFrame(toppool_all).sort_values(by = [f"{sort_by_col}"],
                                                 ascending = False).head(num_rows)

def get_network_token_add_dict(sort_by_col, num_rows):
    
    df_new_tokens = get_new_pools(sort_by_col, num_rows)
    
    return df_new_tokens.groupby("network").agg(list)["token_add"].to_dict() 

In [165]:
def track_status_all(sort_by_col, num_rows):
    
    tokens_dict = get_network_token_add_dict(sort_by_col, num_rows)
    
    dfs = []
    
    for network, list_of_tokens in tokens_dict.items():
        print("Analyzing tokens for network:", network)
        df_status = track_status(list_of_tokens, network, 95)
        dfs.append(df_status)
        
    # Merge into one DataFrame
    df_final = pd.concat(dfs, ignore_index=True)
    
    return df_final

In [170]:
get_new_pools("market_cap_usd", 2)

,pair,dex,network,token_add,pool_add,fdv_usd,market_cap_usd,daily_volume,daily_price_change,daily_buys,daily_sells,daily_buyers,daily_sellers
0,RTJSHFGJDFGG / WBNB,pancakeswap_v2,bsc,0xea525a18561c93589a1e161120c7450b00f32915,0x294c2a993f1e9565dcfe3df134269e0eb9f394e7,1291.108654,None,7.21512,0,1,0,1,0
1,CP / BNB,four-meme,bsc,0x35f999f79285a54c3750c2b0c31bf26497ee4444,0x35f999f79285a54c3750c2b0c31bf26497ee4444,6258.56199,None,1218.6678580133,0.476,3,2,3,2


### Streamlit dashboard